In [25]:
import pandas as pd
import numpy as np
import base64
import openpyxl
from IPython.display import display

# Read file from OneDrive
def create_onedrive_directdownload(onedrive_link):
    data_bytes64 = base64.b64encode(bytes(onedrive_link, 'utf-8'))
    data_bytes64_String = data_bytes64.decode('utf-8').replace('/','_').replace('+','-').rstrip("=")
    resultUrl = f"https://api.onedrive.com/v1.0/shares/u!{data_bytes64_String}/root/content"
    return resultUrl

onedrive_link = "https://1drv.ms/x/s!AosADtCoLH31glcC7hCv82W3ZAUj?e=ZGpMR1"
onedrive_direct_link = create_onedrive_directdownload(onedrive_link)

costs = pd.read_excel(onedrive_direct_link)
    
# Drop empty rows & columns
costs = costs.dropna(how='all', axis=1)
costs = costs.dropna(how='all', axis=0)

# Drop unnamed columns
to_drop = costs.columns[costs.columns.str.startswith('Unnamed')]
costs.drop(to_drop, axis=1, inplace=True)

# Fill empties
costs["Dye_Type"] = costs["Dye_Type"].fillna("")

# Rename the costs for clarity

new_cost_names = []

for cost_name, item_name, uom, dye_type in zip(costs['Cost Name'], costs['Item Name'], costs["UOM"], costs["Dye_Type"]):

    # Add dye info
    if cost_name == "Dye":
        new_cost_names.append(str(cost_name + " - " + dye_type))
    
    # Replace question marks with Item Names for Clarity
    elif "?" in cost_name:
        replacement = "for" + str(item_name)
        new_cost_names.append(cost_name.replace('?', replacement))
    
    # Add the size to the individual sublimation prints
    elif cost_name == "Sublimation Print - Individual":
        new_cost_names.append(str(cost_name + " (" + uom + ")"))
    
    else:
        new_cost_names.append(cost_name)
        
costs["Cost"] = new_cost_names
costs = costs.drop("Cost Name", axis=1)

# Correct the UOMs for sublimation size clarity
new_UOM = []

for uom in costs["UOM"]:
    if "A" in uom:
        new_UOM.append("pcs")
    else:
        new_UOM.append(uom)
        
costs["UOM"] = new_UOM

# Season Filter
season_filter = input("Would you like to filter by season? (y/n)")

if season_filter == "n":
    pass
elif season_filter == "y":
    season_selector = input("Enter M for mainline, R for resort or B for both:")
    if season_selector.lower() == "m":
        costs = costs[costs["Season"] == "YS1"]
        costs = pd.DataFrame(costs)
    elif season_selector.lower() == "r":
        costs = costs[costs["Season"] == "YR1"]
        costs = pd.DataFrame(costs)
    else:
        pass
    
# Material Filter
material_filter = input("Are you calculating for a specific material? (y/n)")

if material_filter == "n":
    pass
elif material_filter == "y":
    material_selector = input("Please copy & paste the material name exactly as it appears on the spreadsheet:")
    costs = costs[costs["Cost"] == material_selector]

# Select materials only
material_costs = costs[costs["Cost Label"] == "Materials"]
material_costs = material_costs[["Item Name", "Cost", "Yield", "UOM", "Supplier", "Cost Per"]]
material_costs.head()

# Costs Cleaning
material_costs["Cost Per"] = [float(i) for i in material_costs["Cost Per"]]

material_costs = material_costs.rename(columns={"Yield": "Units", 
                                                "Cost Per": "Cost Per Unit", 
                                                "Cost" : "Material"})
costs = pd.DataFrame(material_costs)

costs_obj = costs.select_dtypes(['object'])
costs[costs_obj.columns] = costs_obj.apply(lambda x: x.str.strip())

# Get Total Quantities & Print Output

manual_quantity_input_is = input("Please enter '0' if you'd like to use the pre-existing quantities, or '1'if you'd like to enter your own")

if manual_quantity_input_is == "1":
    
    item_names = {}

    for item_name in costs["Item Name"].unique():
        item_names[item_name] = 0

    for key, value in item_names.items():
        item_names[key] = int(input("How many " + key + "?"))
        
    costs['Quantity'] = costs['Item Name'].map(item_names)

    material_costs = costs.copy()

    material_costs["Total Units"] = material_costs["Units"] * material_costs["Quantity"]

    material_costs["Total Cost"] = material_costs["Total Units"] * material_costs["Cost Per Unit"]

    material_costs.head()
    
    print("Your Quantity Request:")

    print("\n")

    for key, value in item_names.items():
        print(str(value) + " " + str(key) + "s")

    print("\n")

    print("This order will cost you approximately", round(material_costs["Total Cost"].sum(),2), "in total")
    print("\n")
    print("Your price breakdown is:")
    
    output_costs = material_costs[['Material', 'Total Units', 'UOM', 'Total Cost', 'Supplier']].groupby('Material', as_index=False).sum(numeric_only = True).round(2)
    
    output_costs.to_excel("output.xlsx") 
    
    display(output_costs)
    
    print("Your output is saved as output.xlsx")

elif manual_quantity_input_is == "0":
    
    onedrive_quantities_link = "https://1drv.ms/x/s!AosADtCoLH31glNXXx7zye1wxIAO?e=3qa5el"
    onedrive_direct_link = create_onedrive_directdownload(onedrive_quantities_link)
    quantities = pd.read_excel(onedrive_direct_link)

    quantities = quantities[["ITEM", "GRAND TOTAL"]][1:]
    
    new_quantities = quantities[["ITEM", "GRAND TOTAL"]].reset_index()
    new_quantities = new_quantities.drop("index", axis=1).set_index("ITEM")

    item_names = new_quantities.to_dict()["GRAND TOTAL"]
    
    costs['Quantity'] = costs['Item Name'].map(item_names)
    
    costs[['Item Name', 'Quantity']].to_excel("test.xlsx") 

    material_costs = costs.copy()

    material_costs["Total Units"] = material_costs["Units"] * material_costs["Quantity"]

    material_costs["Total Cost"] = material_costs["Total Units"] * material_costs["Cost Per Unit"]

    material_costs.head()
    
    print("Your Quantity Request:")

    print("\n")

    for key, value in item_names.items():
        print(str(value) + " " + str(key) + "s")

    print("\n")

    print("This order will cost you approximately", round(material_costs["Total Cost"].sum(),2), "in total")
    print("\n")
    print("Your price breakdown is:")
    
    output_costs = material_costs[['Material', 'Total Units', 'UOM', 'Total Cost', 'Supplier']].groupby('Material', as_index=False).sum(numeric_only = True).round(2)
    
    
    output_costs.to_excel("output.xlsx") 
    
    display(output_costs) 
    
    print("Your output is saved as output.xlsx")

Would you like to filter by season? (y/n)n
Are you calculating for a specific material? (y/n)y
Please copy & paste the material name exactly as it appears on the spreadsheet:Velcro
Please enter '0' if you'd like to use the pre-existing quantities, or '1'if you'd like to enter your own0
Your Quantity Request:


7.0 6-ARM PUFFERs
9.0 10-POC CARGO PANTs
4.0 SSENSE 7-POC CARGO PANT (BLACK)s
3.0 SSENSE 7-POC CARGO PANT (GREEN)s
0.0 ZILLA SKIN BONNETs
10.0 ZILLA SKIN TOPs
8.0 RALIEN HOODIE (GREEN)s
11.0 RALIEN PANT (GREEN)s
5.0 SSENSE RALIEN HOODIE (GREEN)s
5.0 SSENSE RALIEN PANT (GREEN)s
6.0 SSENSE RALIEN HOODIE (BLACK)s
8.0 SSENSE RALIEN PANT (BLACK)s
21.0 RALIEN GLOVESs
43.0 RALIEN TEE (MULTI)s
18.0 SSENSE RALIEN TEE (BLACK)s
11.0 CAEDON BAGs
7.0 S-TOPs
12.0 S-PANTs
7.0 S-GILLETs
9.0 M-JACKETs
3.0 M-PANTs
8.0 M-GILLETs
33.0 ELI-MESH SHIRTs
3.0 ELI COMBAT HATs
5.0 OOPS CUSTOM SHOEs
2.0 MA S-GILLET BLUEs
9.0 MA EXCLUSIVE TEEs
13.0 FS EXCLUSIVE TEEs
10.0 RESORT RALIEN GLOVEs
4.0 RESORT EYE H

,Material,Total Units,Total Cost
0,Velcro,12.8,46.08


Your output is saved as output.xlsx
